# Práctica: "¿Quién dijo qué? – Detectando Autores con IA" ️
### Objetivo
Construiremos un sistema para predecir el autor de un texto basándonos en su estilo de escritura.

### Pasos:
1. Cargar y preprocesar textos de diferentes autores.
2. Representarlos con TF-IDF.
3. Calcular similitudes usando el coseno.
4. Predecir el autor más probable.

---

In [1]:
!pip install -q scikit-learn numpy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

## 1️⃣ Cargar y Preprocesar los Textos

In [17]:
# Ejemplo de textos de diferentes autores

texto_quijote1 = "En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lentejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entre semana se honraba con su vellorí de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera."
texto_quijote2 = "Digo, pues, que, entre otras cosas que tenía descuidadas este hidalgo, era de su ordinario cuidado el de su caballo Rocinante, y así lo tenía en un rincón de un patio, arrimado a una estaca, donde le tenía atado con una cuerda, y, cuando le quería ensillar, le daba una palmada en el trasero y le decía: «¡Ea, Rocinante, ea!», como si el animal entendiera lo que le decía; y, como el pobre animal era de condición sufrida, ni se movía ni hacía ademán alguno, y así lo ensillaba y montaba sin dificultad alguna. Y, cuando quería salir a alguna parte, le decía: «¡Ea, Rocinante, ea!», y el animal, como si entendiera lo que le decía, se ponía en camino sin dificultad alguna."
texto_lazarillo = "Pues, siendo niño de ocho años, achacaron a mi padre ciertas sangrías mal hechas en los costales de la parroquia, y, por ellas, fue preso, y confesó, y no negó, y padeció persecución por justicia. Espero en Dios que está en la gloria, pues el evangelista dice bienaventurados los que padecen persecución por justicia. En este tiempo, se hizo cierta armada contra moros, entre los cuales fue mi padre, que a la sazón estaba desterrado, 1  y con cargo de acemilero de un caballero, allí feneció su vida."
texto_inventado = "Ayer me encontré con un tipo que se creía la última Coca-Cola del desierto. Iba todo flipado, diciendo que era un influencer y que tenía miles de seguidores. Le pregunté si sabía lo que era un libro, y me miró como si le hubiera hablado en chino"

# Preprocesamiento de textos

def minuscula_y_sin_puntuacion(texto):
    texto = texto.lower()
    # Eliminar puntuación y caracteres especiales utilizando expresiones regulares
    texto = re.sub(r'[^\w\s]', ' ', texto)
    return texto

texto_quijote1 = minuscula_y_sin_puntuacion(texto_quijote1)
texto_quijote2 = minuscula_y_sin_puntuacion(texto_quijote2)
texto_lazarillo = minuscula_y_sin_puntuacion(texto_lazarillo)
texto_inventado = minuscula_y_sin_puntuacion(texto_inventado)





## 2️⃣ Representación con TF-IDF

In [18]:
from os import PRIO_USER
# Vectorización con TF-IDF
# Vectorización con TF-IDF
vectorizer = TfidfVectorizer()
matriz_tfidf_quijote = vectorizer.fit_transform([texto_quijote1, texto_quijote2])
matriz_tfidf_quijote_lazarillo = vectorizer.fit_transform([texto_quijote1, texto_lazarillo])
matriz_tfidf_inventado = vectorizer.fit_transform([texto_quijote1, texto_inventado])

# La matriz TF-IDF resultante
print("Matriz dos textos del Quijote\n")
print(matriz_tfidf_quijote.toarray())
print("\nMatriz un texto del Quijote y un texto de lazarillo\n")
print(matriz_tfidf_quijote_lazarillo.toarray())
print("\nMatriz un texto del Quijote y un texto inventado\n")
print(matriz_tfidf_inventado.toarray())



Matriz dos textos del Quijote

[[0.05885171 0.05885171 0.         0.05885171 0.         0.
  0.05885171 0.05885171 0.         0.05885171 0.         0.05885171
  0.04187347 0.         0.05885171 0.         0.05885171 0.
  0.05885171 0.05885171 0.05885171 0.04187347 0.08374695 0.05885171
  0.         0.05885171 0.05885171 0.         0.         0.05885171
  0.         0.         0.05885171 0.         0.58622863 0.
  0.05885171 0.         0.         0.         0.05885171 0.
  0.05885171 0.05885171 0.         0.08374695 0.12562042 0.04187347
  0.         0.         0.04187347 0.         0.         0.
  0.05885171 0.05885171 0.05885171 0.05885171 0.05885171 0.05885171
  0.         0.04187347 0.05885171 0.11770341 0.05885171 0.17655512
  0.         0.05885171 0.05885171 0.08374695 0.41196194 0.05885171
  0.05885171 0.05885171 0.         0.         0.05885171 0.05885171
  0.17655512 0.         0.17655512 0.05885171 0.05885171 0.05885171
  0.         0.         0.         0.05885171 0.05885171 

## 3️⃣ Cálculo de Similitudes

In [19]:
# Función de similitud de coseno

# Vectorización con TF-IDF,  incluyendo todos los textos en la misma matriz
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform([texto_quijote1, texto_quijote2, texto_lazarillo, texto_inventado])

# Cálculo de la similitud del coseno utilizando la matriz con todos los textos
similitud_quijote1_quijote2 = cosine_similarity(matriz_tfidf[0], matriz_tfidf[1])[0][0]
similitud_quijote1_lazarillo = cosine_similarity(matriz_tfidf[0], matriz_tfidf[2])[0][0]
similitud_quijote1_inventado = cosine_similarity(matriz_tfidf[0], matriz_tfidf[3])[0][0]

print(f"Similitud del coseno entre Quijote 1 y Quijote 2: {similitud_quijote1_quijote2}")
print(f"Similitud del coseno entre Quijote 1 y Lazarillo: {similitud_quijote1_lazarillo}")
print(f"Similitud del coseno entre Quijote 1 y Inventado: {similitud_quijote1_inventado}")



Similitud del coseno entre Quijote 1 y Quijote 2: 0.23004393042359342
Similitud del coseno entre Quijote 1 y Lazarillo: 0.3193134424479354
Similitud del coseno entre Quijote 1 y Inventado: 0.16571530737289317


# Conclusiones
Aunque curiosamente el primer texto del Quijote se parece más a uno extraído del Lazarillo de Tormes que a otro texto del propio Quijote, se ve la diferencia con un texto generado automáticamente con un lenguaje actual.